# GREAT CARIA v4: UNIFIED FIELD THEORY (Final)

## The Synthesis of Empirical Power and Theoretical Elegance

**Objective:** Restore peak predictive performance (AUC > 0.705) while maintaining the "General Financial Relativity" theoretical framework.

### The Unified Field Equation $\Psi(t)$

We define the Market Stress Wavefunction as the ratio of **Systemic Mass** (Consensus) to **Spacetime Uncertainty** (Dispersion), modulated by **Geographic Spin**.

$$ \Psi(t) = \frac{\mu(t) \times \text{Spin}(t)}{\sqrt{H(t)^2 + \hbar^2}} $$

Where:
1.  **Mass ($\mu$)**: The **Crisis Factor (CF)**. Empirically the strongest signal (Correlation $\times$ Volatility). Represents the "gravitational pull" of the market crash.
2.  **Uncertainty ($H$)**: **3-Band Dispersion**. The divergence between Fast, Medium, and Slow temporal scales. Low $H$ = "Synchronization" (Danger).
3.  **Spin**: **Geographic Relativity**. A risk multiplier. If G7 volatility > EM volatility, the system is fundamentally more unstable (Spin > 1.0).
4.  **$\hbar$**: Financial Planck Constant. The minimum uncertainty limit to prevent singularities.

### The Event Horizon
The "Event Horizon" is visualized when the **Probability Cone** (defined by $\mu \pm k \cdot H$) narrows below the Schwarzschild Radius ($R_s$). This corresponds to a state of **Critical Synchronization** where alternative futures collapse into a single crisis trajectory.

In [ ]:
# === SETUP ===
!pip install PyWavelets scikit-learn numpy pandas scipy matplotlib -q

import pandas as pd
import numpy as np
from scipy import stats, signal
import pywt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import warnings
import json
from datetime import datetime

warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

print('='*80)
print('GREAT CARIA v4: UNIFIED FIELD THEORY')
print('Synthesizing Empirical CF with Relativistic H')
print('='*80)

In [ ]:
# === LOAD DATA ===
DATA_PATH = '/content/drive/MyDrive/Caria/yahoo_market.parquet'
try:
    df = pd.read_parquet(DATA_PATH)
except FileNotFoundError:
    # Fallback path if folder structure differs
    DATA_PATH = '/content/drive/MyDrive/CARIA/data/raw/yahoo_market.parquet'
    df = pd.read_parquet(DATA_PATH)

df.index = pd.to_datetime(df.index)

# Geographic Blocks
G7_COUNTRIES = ['USA', 'GBR', 'DEU', 'FRA', 'JPN', 'CAN']
EM_COUNTRIES = ['CHN', 'BRA', 'IND', 'MEX', 'KOR', 'ZAF', 'AUS'] # adding AUS to EM block for commodity sensitivity

# Extract Indices
all_cols = [c for c in df.columns if '_index' in c]
ret = df[all_cols].pct_change().dropna()
ret.columns = [c.replace('_index', '') for c in ret.columns]

# Separate Blocks
g7_cols = [c for c in G7_COUNTRIES if c in ret.columns]
em_cols = [c for c in EM_COUNTRIES if c in ret.columns]

ret_g7 = ret[g7_cols]
ret_em = ret[em_cols]

print(f'Data: {ret.shape[0]} days')
print(f'G7 Block: {g7_cols}')
print(f'EM Block: {em_cols}')

In [ ]:
# === GROUND TRUTH: CRISIS CATALOG ===
CRISES = {
    'Lehman': pd.Timestamp('2008-09-15'),
    'Flash_Crash': pd.Timestamp('2010-05-06'),
    'Euro_Crisis': pd.Timestamp('2011-08-05'),
    'China_Crash': pd.Timestamp('2015-08-24'),
    'Brexit': pd.Timestamp('2016-06-24'),
    'Volmageddon': pd.Timestamp('2018-02-05'),
    'COVID': pd.Timestamp('2020-03-11'),
    'Gilt_Crisis': pd.Timestamp('2022-09-23'),
    'SVB': pd.Timestamp('2023-03-10')
}

# Filter crises in data range
CRISES = {k: v for k, v in CRISES.items() if v >= ret.index.min() and v <= ret.index.max()}
print(f'Valid Crises for Validation: {len(CRISES)}')
for k, v in CRISES.items():
    print(f'  - {k}: {v.date()}')

# Binary Target for AUC
def create_crisis_labels(index, crises, pre_window=20, post_window=10):
    labels = pd.Series(0, index=index)
    for name, date in crises.items():
        start = date - pd.Timedelta(days=pre_window)
        end = date + pd.Timedelta(days=post_window)
        labels[(labels.index >= start) & (labels.index <= end)] = 1
    return labels

y_true = create_crisis_labels(ret.index, CRISES)

---
## Part 1: Systemic Mass ($\mu$) - The Crisis Factor
Total market stress is defined by mass $\mu$, which is the product of **Correlation** and **Volatility**.
High $\mu$ means the market is both moving violently AND moving together (heavy object falling).

In [ ]:
def compute_cf(r, window=20):
    """Crisis Factor (Mass) = avg_correlation * avg_volatility"""
    cf = []
    for i in range(window, len(r)):
        wr = r.iloc[i-window:i]
        corr_matrix = wr.corr().values
        n = len(corr_matrix)
        # Average off-diagonal correlation
        avg_corr = (corr_matrix.sum() - n) / (n * (n - 1))
        # Average volatility
        avg_vol = wr.std().mean()
        # CF Signal
        cf.append(avg_corr * avg_vol * 100)
    return pd.Series(cf, index=r.index[window:])

MU = compute_cf(ret)
# Normalize Mass [0, 1] for physics equations
MU_NORM = (MU - MU.min()) / (MU.max() - MU.min())

print(f'Systemic Mass (CF) computed. Mean: {MU.mean():.4f}')

---
## Part 2: Uncertainty ($H$) - 3-Band Dispersion
Uncertainty corresponds to the **Dispersion** of market signals across time scales.
- **High $H$**: Signals are scattered. Fast, Medium, and Slow traders disagree. (Stable)
- **Low $H$**: Signals align. Synchronization across timescales. (Danger)

We use wavelet decomposition (Slow, Medium, Fast) and calculate the standard deviation between them.

In [ ]:
def compute_uncertainty_h(series, wavelet='db4'):
    """H = Dispersion between temporal bands"""
    # Decompose
    coeffs = pywt.wavedec(series.values, wavelet, level=6)
    
    # Reconstruct 3 Bands
    # Fast: levels 0-2
    fast = pywt.waverec([coeffs[0], coeffs[1], coeffs[2]] + [np.zeros_like(c) for c in coeffs[3:]], wavelet)[:len(series)]
    # Medium: levels 3-4
    med = pywt.waverec([np.zeros_like(c) for c in coeffs[:3]] + [coeffs[3], coeffs[4]] + [np.zeros_like(c) for c in coeffs[5:]], wavelet)[:len(series)]
    # Slow: levels 5+
    slow = pywt.waverec([np.zeros_like(c) for c in coeffs[:5]] + [coeffs[5], coeffs[6]], wavelet)[:len(series)]
    
    # Calculate Dispersion (Standard Deviation across bands at each t)
    bands = np.vstack([fast, med, slow])
    dispersion = np.std(bands, axis=0)
    
    return pd.Series(dispersion, index=series.index)

H = compute_uncertainty_h(MU_NORM)

# Planck Constant (min uncertainty)
H_BAR = H.quantile(0.05)
print(f'Uncertainty (H) computed. Planck Constant (h_bar): {H_BAR:.4f}')

---
## Part 3: Geographic Spin - The Risk Multiplier
Relativity dictates that the observer's frame matters. In global finance, the stability of the G7 Core is the reference frame.
- **Normal State**: EM Volatility > G7 Volatility. (Spin $\approx$ 1.0). Risk is standard.
- **Crisis State**: G7 Volatility > EM Volatility. (Spin > 1.0). The core is unstable. Risk is multiplied.

$$ \text{Spin}(t) = 1 + \alpha \cdot \max(0, \sigma^2_{G7} - \sigma^2_{EM}) $$

In [ ]:
def compute_spin(g7, em, window=60, alpha=5.0):
    """Calculate Spin Multiplier"""
    vol_g7 = g7.std(axis=1).rolling(window).mean()
    vol_em = em.std(axis=1).rolling(window).mean()
    
    # Difference (G7 - EM)
    diff = vol_g7 - vol_em
    
    # Only penalize if G7 > EM (diff > 0)
    # We normalize diff to be meaningful
    # We assume 'alpha' is the sensitivity parameter
    
    spin_term = diff.apply(lambda x: max(0, x))
    
    # Spin starts at 1.0 and increases when G7 is unstable
    # We scale spin_term by a factor to make it impactful (e.g. 10x the raw var difference)
    spin = 1 + (spin_term * alpha)
    
    return spin

# Use smaller window for spin to capture shock
SPIN = compute_spin(ret_g7, ret_em, window=40, alpha=20.0)
SPIN = SPIN.reindex(MU.index).fillna(1.0)

print(f'Geographic Spin computed. Max Spin: {SPIN.max():.2f} (during G7 instability)')

---
## Part 4: The Unified Field $\Psi(t)$ and Event Horizon

$$ \Psi(t) = \frac{\mu(t) \cdot \text{Spin}(t)}{\sqrt{H(t)^2 + \hbar^2}} $$

- The denominator $\sqrt{H^2 + \hbar^2}$ represents the "Lorentzian width" of the probability cone.
- As $H \to 0$, the cone narrows. $\Psi$ spikes.
- $\text{Spin}$ amplifies the mass when the G7 core is shaking.

In [ ]:
# === CALCULATE PSI ===
denominator = np.sqrt(H**2 + H_BAR**2)
PSI = (MU_NORM * SPIN) / denominator

# Smooth slightly to remove daily noise
PSI = PSI.rolling(5).mean()

# Replace infs if any
PSI = PSI.replace([np.inf, -np.inf], np.nan).fillna(method='ffill')

print(f'Psi Calculated. Range: {PSI.min():.2f} to {PSI.max():.2f}')

# === EVENT HORIZON CONDITION ===
# Horizon is breached when Uncertainty is critically low relative to Mass
# We define 'Cone Width' as H(t)
# Schwarzschild Radius (Rs) is the critical width

Rs = H.quantile(0.10) # 10th percentile of uncertainty

EVENT_HORIZON = (H < Rs).astype(int)
print(f'Event Horizon Radius (Rs): {Rs:.4f}')
print(f'Time inside Event Horizon: {EVENT_HORIZON.sum()} days ({EVENT_HORIZON.mean()*100:.1f}%)')

---
## Part 5: Validation

In [ ]:
# Align Data
valid_idx = y_true.index.intersection(PSI.dropna().index)
y_test = y_true.loc[valid_idx]
psi_test = PSI.loc[valid_idx]
h_test = H.loc[valid_idx]
mu_test = MU_NORM.loc[valid_idx]

# Baseline: Raw Volatility
vol_test = ret.std(axis=1).rolling(20).mean().loc[valid_idx]

# Compare Models
results = {}
for name, score in [
    ('Great Caria v4 (Psi)', psi_test),
    #('Event Horizon Only (1/H)', 1/(h_test + 0.001)), # Simple inverse H
    ('Mass Only (CF)', mu_test),
    ('Volatility (Baseline)', vol_test)
]:
    fpr, tpr, _ = roc_curve(y_test, score.fillna(0))
    roc_auc = auc(fpr, tpr)
    results[name] = roc_auc
    
print('=== PREDICTIVE POWER (AUC) ===')
for name, res in sorted(results.items(), key=lambda x: -x[1]):
    print(f'{name:25s}: {res:.3f}')

# Plot ROC
plt.figure(figsize=(8, 6))
for name in results.keys():
    score = [psi_test, 1/h_test, mu_test, vol_test]
    # Re-calc for plot loop simplicity
    s = psi_test if name == 'Great Caria v4 (Psi)' else (mu_test if name == 'Mass Only (CF)' else vol_test)
    fpr, tpr, _ = roc_curve(y_test, s.fillna(0))
    plt.plot(fpr, tpr, label=f'{name} (AUC={results[name]:.3f})')
plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Great Caria v4 Validation')
plt.legend(loc='lower right')
plt.show()

---
## Part 6: Relativistic Visualization (The Cone)

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

# 1. Mass and Spin
ax1 = axes[0]
ax1.plot(MU.index, MU_NORM, color='blue', label='Mass (CF)', alpha=0.7)
ax1.plot(SPIN.index, (SPIN-1), color='purple', label='Spin Bonus (G7 Risk)', alpha=0.5)
ax1.set_title('Component 1: Mass & Spin')
ax1.legend()

# 2. The Probability Cone (Uncertainty)
ax2 = axes[1]
mean_line = pd.Series(0, index=H.index)
# Cone Boundary
ax2.fill_between(H.index, -H, H, color='green', alpha=0.2, label='Probability Cone (W = 2H)')
# Event Horizon Zone (Black)
horizon_mask = H < Rs
ax2.fill_between(H.index, -0.5, 0.5, where=horizon_mask, color='black', alpha=0.3, label='Event Horizon (Collapsed)')
ax2.set_ylim(-0.5, 0.5)
ax2.set_title(f'Component 2: Spacetime Uncertainty (Event Horizon at W < {Rs:.3f})')
ax2.legend(loc='upper right')

# 3. Final Wavefunction Psi
ax3 = axes[2]
ax3.plot(PSI.index, PSI, color='red', linewidth=1.5, label='Psi(t) - Collapse Probability')
# Thresholds
thresh_crit = PSI.quantile(0.95)
thresh_warn = PSI.quantile(0.80)
ax3.axhline(thresh_crit, color='darkred', linestyle='--', label='Critical')
ax3.axhline(thresh_warn, color='orange', linestyle='--', label='Warning')

ax3.set_title(f'Great Caria v4: Unified Field (AUC = {results["Great Caria v4 (Psi)"]:.3f})')
ax3.legend()

# Mark Crises
for ax in axes:
    for name, date in CRISES.items():
        ax.axvline(date, color='black', linestyle=':', alpha=0.5)
        if ax == axes[0]:
            ax.text(date, 1.05, name, rotation=90, verticalalignment='bottom', fontsize=8)

plt.tight_layout()
plt.savefig('/content/drive/MyDrive/Caria/great_caria_v4_unified.png', dpi=150)
plt.show()

In [ ]:
# === EXPORT ===
latest_date = df.index[-1]
current_psi = PSI.iloc[-1]
current_h = H.iloc[-1]
in_horizon = current_h < Rs

status = "STABLE"
if current_psi > thresh_crit: status = "CRITICAL"
elif current_psi > thresh_warn: status = "WARNING"

export_data = {
    "version": "v4.0 (Unified Field)",
    "generated_at": datetime.now().isoformat(),
    "last_market_date": latest_date.strftime('%Y-%m-%d'),
    "status": status,
    "metrics": {
        "psi": float(current_psi),
        "mass_cf": float(MU.iloc[-1]),
        "uncertainty_h": float(current_h),
        "spin": float(SPIN.iloc[-1])
    },
    "event_horizon": {
        "is_inside": bool(in_horizon),
        "schwarzschild_radius": float(Rs),
        "current_width": float(current_h)
    },
    "thresholds": {
        "warning": float(thresh_warn),
        "critical": float(thresh_crit)
    },
    "auc_score": float(results['Great Caria v4 (Psi)']),
    "history": {
        "dates": [d.strftime('%Y-%m-%d') for d in PSI.index[-100:]],
        "psi": PSI.iloc[-100:].tolist(),
        "upper_cone": H.iloc[-100:].tolist(),
        "lower_cone": (-H.iloc[-100:]).tolist()
    }
}

with open('/content/drive/MyDrive/Caria/great_caria_v4.json', 'w') as f:
    json.dump(export_data, f, indent=2)

print("Exported v4 JSON successfully.")